Data for US confirmed cases and deaths can be found at the following links. Worldwide data is also available in the same repository.

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv


Download US sequential data:

In [91]:
import urllib.request
import os

# URLs
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
counties_map_url = 'https://upload.wikimedia.org/wikipedia/commons/5/5f/USA_Counties_with_FIPS_and_names.svg'


# File names
confirmed_file_name = confirmed_url.split('/')[-1]
deaths_file_name = deaths_url.split('/')[-1]
counties_map_file_name = counties_map_url.split('/')[-1]


# Folders
base_folder = os.getcwd()
download_folder = base_folder + '/data/'
map_folder = base_folder + '/map/'

if not os.path.exists(download_folder):
    os.makedirs(download_folder)
    
if not os.path.exists(map_folder):
    os.makedirs(map_folder)


# Communication "The Art Of Communication Is The Language Of Leadership." Wow, I'm inspired.
print('Fetching {} ...'.format(confirmed_file_name), end='')
urllib.request.urlretrieve(confirmed_url, download_folder + confirmed_file_name)
print('done')

print('Fetching {} ...'.format(deaths_file_name), end='')
urllib.request.urlretrieve(deaths_url, download_folder + deaths_file_name)
print('done')

print('Fetching {} ...'.format(counties_map_file_name), end='')
urllib.request.urlretrieve(counties_map_url, map_folder + counties_map_file_name)
print('done')


Fetching time_series_covid19_confirmed_US.csv ...done
Fetching time_series_covid19_deaths_US.csv ...done
Fetching USA_Counties_with_FIPS_and_names.svg ...done


#### Put the data from the csv into a numpy array


# Old

In [51]:
import csv
import numpy as np


file = download_folder + confirmed_file_name
print("Opening " + file + '\n')

# open the csv file
with open(file, newline='') as csvfile:
    covid_confirmed_us = csv.reader(csvfile, delimiter=',')
    
    # iterate through the headers file, but save it
    headers_confirmed = next(covid_confirmed_us)
    row_length = len(headers_confirmed)
    
    # get headers before first day of data (11)
    #for i, val in enumerate(headers):
    #    print("{}: {}".format(i,val))
    first_day_idx = 11                                  # index of first day in row
    FIPS_idx = 4                                        # FIPS code is a county code used to edit map
    num_days = row_length-first_day_idx                 # number of days of data     
    confirmed_data = np.empty((0,num_days+1), int)      # rows = counties,  cols = [id, day0, day1...]
    
    
    # Wave the data to matrix. One county per row
    for i, row in enumerate(covid_confirmed_us):
        
        temp_row = np.zeros((1, num_days+1), int)
        
        try:
            temp_row[0][0] = int(float(row[FIPS_idx]))                # place county id in 0th index
        except:
            print("uh yeah no. Nice try, data at ({}, {})".format(i, FIPS_idx))
        
        
        temp_row[0][1:] = row[first_day_idx:]                         # copy days data to temp, after county id
        confirmed_data = np.append(confirmed_data, temp_row, axis=0)  # add the temp_row as a new row
        
           
    
    print(confirmed_data)                                             # confirm everything looks a-ok


Opening /Users/ethanblagg/SynologyDrive/Documents/Projects/YT/Covid/data/time_series_covid19_confirmed_US.csv

uh yeah no. Nice try, data at (3147, 4)
uh yeah no. Nice try, data at (3148, 4)
[[   60     0     0 ...     0     0     0]
 [   66     0     0 ...   121   128   130]
 [   69     0     0 ...     6     6     6]
 ...
 [90055     0     0 ...     0     0     0]
 [90056     0     0 ...     0     0     0]
 [99999     0     0 ...   103   103   103]]


# New

In [77]:
import csv
import numpy as np


file = download_folder + confirmed_file_name
print("Opening " + file + '\n')

# open the csv file
with open(file, newline='') as csvfile:
    covid_confirmed_us = csv.reader(csvfile, delimiter=',')
    
    # iterate through the headers file, but save it
    headers_confirmed = next(covid_confirmed_us)
    row_length = len(headers_confirmed)
    
    # get headers before first day of data (11)
    #for i, val in enumerate(headers):
    #    print("{}: {}".format(i,val))
    first_day_idx = 11                                  # index of first day in row
    FIPS_idx = 4                                        # FIPS code is a county code used to edit map
    num_days = row_length-first_day_idx                 # number of days of data     
    confirmed_data = np.zeros((100000,num_days), int) # rows = counties,  cols = [id, day0, day1...]
    
    
    # Move the data to matrix. One county per row
    for i, row in enumerate(covid_confirmed_us):
        
        #temp_row = np.zeros((1, num_days), int)
        
        try:
            idx = int(float(row[FIPS_idx]))                # place county id in 0th index
            #temp_row[0][:] = row[first_day_idx:]                         # copy days data to temp, after county id
            confirmed_data[idx][:] = row[first_day_idx:]
        except:
            print("Can't find FIPS data for {}".format(row[10]))
        
        
           
    
    print(confirmed_data[66])                                             # confirm everything looks a-ok
    print(confirmed_data[99999])


Opening /Users/ethanblagg/SynologyDrive/Documents/Projects/YT/Covid/data/time_series_covid19_confirmed_US.csv

Can't find FIPS data for Dukes and Nantucket,Massachusetts,US
Can't find FIPS data for Kansas City,Missouri,US
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   3   3   5  12  14  15  27  29  32  37  45  51  55  56  58  69  77  82
  84  93 112 113 121 121 128 130]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  20  21  21  22  23  23  30  28  28  28  28  28 103 103 103 103 103 103
 103 103 103 103 103 103 103 103]


In [90]:
from bs4 import BeautifulSoup
import time

#out_file = 'Covid_confirmed_map_' + time.strftime("%y-%m-%d") + ".svg"
out_file = 'We\'re_all_gonna_die_' + time.strftime("%y-%m-%d") + ".svg"

svg = open(map_folder + counties_map_file_name, 'r').read()    # Load the SVG map
soup = BeautifulSoup(svg)                                      # Load into Beautiful Soup
paths = soup.findAll('path')                                   # Find counties


# display rules for each county. Note fill value will be added later, according to data
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start:none;stroke-linejoin:bevel;fill:'


for p in paths:
     
    if p['id'] not in ["State_Lines", "separator"]:
        
        try:
            idx = int(p['id'].split("_")[-1])
            #print(idx)
        except:
            continue
        
        
        color = "#d0d0d0"                                              # default if no data
        
        colors = ["#66ffff","#66ff66","#ffff66","#ffb366","#ff6666"]   # alarming, yet calming
        #colors = ["#b3ffff","#ccffcc","#ffff99","#ffcc99","#ff9999"]  # wussy
        #colors = ["#00FFFF","#00FF00","#FFFF00","#FF7F00","#FF0000"]  # bold
        
        
        if (confirmed_data[idx][num_days-1] < 1):
            color = colors[0]
            
        elif (confirmed_data[idx][num_days-1] < 10):
            color = colors[1]
            
        elif (confirmed_data[idx][num_days-1] < 100):
            color = colors[2]
            
        elif (confirmed_data[idx][num_days-1] < 1000):
            color = colors[3]
            
        else:
            color = colors[4]
                                     
        p['style'] = path_style + color

        
#print(soup.prettify())


f = open(map_folder + out_file, 'w')
f.write(soup.prettify())
f.close()
